#  Importing Required modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
import optuna

## Reading data

In [2]:
train_data = pd.read_csv("../input/30-days-of-ml/train.csv")
test_data = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

## Making K-fold model

In [3]:
train_data["kfold"] = -1
Kf_model = KFold(n_splits=5, random_state=1, shuffle=True)
# fold (0, 1, 2, 3, 4); train_index (0, 2, 3...); valid_index(1, 4, 6)
for fold, (train_index, valid_index) in enumerate(Kf_model.split(X=train_data)):
    train_data.loc[valid_index, "kfold"] = fold

In [4]:
print(train_data.shape)
train_data.head()

(300000, 27)


,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0


### Setting required columns

In [5]:
num_cols = [col for col in train_data.columns if "cont" in col]
cat_cols = [col for col in train_data.columns if "cat" in col]
useful_cols = cat_cols + num_cols
test_data = test_data[useful_cols]

## Target Encoding

In [6]:
# Iterate over categorical columns
for col in cat_cols:
    """
    Based on each categorical column, one target fold is created
    total 5 folds for one column
    """
    temp_train = []
    temp_test_target = None
    for fold in range(5):
        # making training data and validating data for each fold
        X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
        X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
        
        # getting the mean of training data target
        mean_target = dict(X_train.groupby(col)["target"].agg("median"))
        
        # adding the mean_target to X_valid (valid dataset of each fold)
        X_valid.loc[:, f"target_enc_{col}"] = X_valid[col].map(mean_target)
        temp_train.append(X_valid)
        if (temp_test_target is None):
            temp_test_target = test_data[col].map(mean_target)
        else:
            temp_test_target += test_data[col].map(mean_target)
        
    # getting the average of temporary test target on each column
    temp_test_target /= 5
    # adding the temporary test target to test data on each column (total=10)
    test_data.loc[:, f"target_enc_{col}"] = temp_test_target
    
    # setting training data as temp_train
    train_data = pd.concat(temp_train)

### Setting required columns again

In [7]:
num_cols = [col for col in train_data.columns if ((train_data[col].dtypes == "int64") or (train_data[col].dtypes =="float64")) and (col not in ("id"))]
cat_cols = [col for col in train_data.columns if (train_data[col].dtypes == "object")]
useful_cols = cat_cols + num_cols
train_data = train_data[useful_cols]
test_data = test_data[useful_cols[:24] + useful_cols[26:]]

In [8]:
print(train_data.columns)
train_data.head()

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6',
       'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13',
       'target', 'kfold', 'target_enc_cat0', 'target_enc_cat1',
       'target_enc_cat2', 'target_enc_cat3', 'target_enc_cat4',
       'target_enc_cat5', 'target_enc_cat6', 'target_enc_cat7',
       'target_enc_cat8', 'target_enc_cat9'],
      dtype='object')


,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,target_enc_cat0,target_enc_cat1,target_enc_cat2,target_enc_cat3,target_enc_cat4,target_enc_cat5,target_enc_cat6,target_enc_cat7,target_enc_cat8,target_enc_cat9
0,B,B,A,A,B,D,A,F,A,O,...,8.200139,8.162371,8.192867,8.234425,8.190434,8.172714,8.19078,8.234335,8.143371,8.175322
1,A,A,A,C,B,D,A,E,A,N,...,8.187044,8.220510,8.192867,8.185586,8.190434,8.172714,8.19078,8.192015,8.143371,8.213467
2,B,A,A,A,B,D,A,E,C,F,...,8.200139,8.220510,8.192867,8.234425,8.190434,8.172714,8.19078,8.192015,8.246532,8.207318
3,A,B,A,C,B,D,A,E,G,I,...,8.187044,8.162371,8.192867,8.185586,8.190434,8.172714,8.19078,8.192015,8.184368,8.159815
4,B,A,A,C,B,D,A,E,A,M,...,8.200139,8.220510,8.192867,8.185586,8.190434,8.172714,8.19078,8.192015,8.143371,8.184239


In [9]:
print(test_data.columns)
test_data.head()

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6',
       'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13',
       'target_enc_cat0', 'target_enc_cat1', 'target_enc_cat2',
       'target_enc_cat3', 'target_enc_cat4', 'target_enc_cat5',
       'target_enc_cat6', 'target_enc_cat7', 'target_enc_cat8',
       'target_enc_cat9'],
      dtype='object')


,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,target_enc_cat0,target_enc_cat1,target_enc_cat2,target_enc_cat3,target_enc_cat4,target_enc_cat5,target_enc_cat6,target_enc_cat7,target_enc_cat8,target_enc_cat9
0,B,B,B,C,B,B,A,E,E,I,...,8.199411,8.162038,8.186756,8.185477,8.189992,8.199043,8.190005,8.191722,8.145356,8.158563
1,A,B,A,C,B,C,A,E,C,H,...,8.186798,8.162038,8.192303,8.185477,8.189992,8.221418,8.190005,8.191722,8.248797,8.195199
2,B,A,A,A,B,B,A,E,D,K,...,8.199411,8.219706,8.192303,8.232115,8.189992,8.199043,8.190005,8.191722,8.205808,8.232337
3,B,B,A,C,B,D,A,E,A,N,...,8.199411,8.162038,8.192303,8.185477,8.189992,8.171351,8.190005,8.191722,8.134364,8.210000
4,B,B,A,C,B,C,A,E,C,F,...,8.199411,8.162038,8.192303,8.185477,8.189992,8.221418,8.190005,8.191722,8.248797,8.206115


## Function on which optuna happen

In [10]:
def optuna_rmse(trial):
   
    fold = 0
    
    # making parameteres
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.5, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    
    # making training data and validating data for each fold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[test_data.columns]
    X_valid = X_valid[test_data.columns]
    
    ## One Hot Encoding for categorical data
    OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
    
    # fitting and transforming the training and test data
    OH_encoder.fit(X_train[cat_cols])
    OH_X_train = OH_encoder.transform(X_train[cat_cols])
    OH_X_valid = OH_encoder.transform(X_valid[cat_cols])
    
    # Naming the one hot encoded columns
    OH_X_train = pd.DataFrame(OH_X_train, columns=[f"ohe_{i}" for i in range(OH_X_train.shape[1])])
    OH_X_valid = pd.DataFrame(OH_X_valid, columns=[f"ohe_{i}" for i in range(OH_X_valid.shape[1])])
    
    # Adding one hot encoded columns to main data (training, validating, test)
    X_train = pd.concat([X_train, OH_X_train], axis=1)
    X_valid = pd.concat([X_valid, OH_X_valid], axis=1)
    
    # Dropping the categorical columns, as their one hot encoded columns are added to main data
    X_train = X_train.drop(cat_cols, axis=1)
    X_valid = X_valid.drop(cat_cols, axis=1)
    
    # making model
    model = XGBRegressor(
        n_estimators=5000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        random_state=50,
    )
    model.fit(X_train, y_train, early_stopping_rounds=300,
              eval_set=[(X_valid, y_valid)], verbose=1000)
    valid_preds = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, valid_preds, squared=False)
    return rmse

## Optuna making and optimizing with `optuna_mse()` function

In [11]:
study_optuna = optuna.create_study(direction="minimize")
study_optuna.optimize(optuna_rmse, n_trials=10)

[I 2021-08-31 19:26:28,942] A new study created in memory with name: no-name-109679a8-0b1b-44b4-9207-b771abc02ec3


[0]	validation_0-rmse:7.38964
[872]	validation_0-rmse:0.72175


[I 2021-08-31 19:26:42,500] Trial 0 finished with value: 0.7207241905976591 and parameters: {'learning_rate': 0.05006097040264347, 'reg_lambda': 2.1469286226544855e-08, 'reg_alpha': 6.290091952555375e-06, 'subsample': 0.22415362332891858, 'colsample_bytree': 0.16220738623925435, 'max_depth': 7}. Best is trial 0 with value: 0.7207241905976591.


[0]	validation_0-rmse:6.50927
[463]	validation_0-rmse:0.72243


[I 2021-08-31 19:26:50,403] Trial 1 finished with value: 0.7192688725248794 and parameters: {'learning_rate': 0.16444552764643636, 'reg_lambda': 44.039623257969474, 'reg_alpha': 1.2931977085976182e-05, 'subsample': 0.7211528911530546, 'colsample_bytree': 0.26320669117639517, 'max_depth': 7}. Best is trial 1 with value: 0.7192688725248794.


...

[I 2021-08-31 19:29:16,371] Trial 7 finished with value: 0.7207344919016684 and parameters: {'learning_rate': 0.019525740726158508, 'reg_lambda': 0.00014273137496083406, 'reg_alpha': 6.072467748725174e-06, 'subsample': 0.7117229439611081, 'colsample_bytree': 0.7309109215968904, 'max_depth': 10}. Best is trial 5 with value: 0.7164083984560765.


[0]	validation_0-rmse:6.68183
[395]	validation_0-rmse:0.73402


[I 2021-08-31 19:29:32,774] Trial 8 finished with value: 0.7235011614085046 and parameters: {'learning_rate': 0.14197762365544714, 'reg_lambda': 0.018089179501302163, 'reg_alpha': 3.667030728747912, 'subsample': 0.6707190861567056, 'colsample_bytree': 0.7251005997193748, 'max_depth': 9}. Best is trial 5 with value: 0.7164083984560765.


[0]	validation_0-rmse:7.54902
[1000]	validation_0-rmse:0.72527
[2000]	validation_0-rmse:0.72145
[3000]	validation_0-rmse:0.71968
[4000]	validation_0-rmse:0.71862
[4999]	validation_0-rmse:0.71792


[I 2021-08-31 19:29:52,763] Trial 9 finished with value: 0.7178961099687168 and parameters: {'learning_rate': 0.02935359063484761, 'reg_lambda': 0.01699697073818512, 'reg_alpha': 0.015605774804323479, 'subsample': 0.41540185161344434, 'colsample_bytree': 0.7904000141284389, 'max_depth': 2}. Best is trial 5 with value: 0.7164083984560765.


## Best Parameters for XGBoost Regressor after optimizing optuna with `optuna_run()` function
#### These parameters with `n_estimators=5000`, `random_state=50` on XGBoostRegressor
##### Some more parameters while fitting the data:
* `early_stopping_rounds=300`
* `verbose=1000`

In [12]:
best_parameters = study_optuna.best_params
best_parameters

{'learning_rate': 0.2591713070909918,
 'reg_lambda': 0.6970000951007623,
 'reg_alpha': 42.03239833403357,
 'subsample': 0.8917988131357745,
 'colsample_bytree': 0.17296908420418067,
 'max_depth': 3}

### Some more parameters with their respective n estimators and rest parameters are:
#### XGBoost Regressor parameters:
* `n_estimators=5000`
* `random_state=50`
##### Some more parameters while fitting the data:
* `early_stopping_rounds=300`
* `verbose=1000`

In [13]:
# n_estimators = 1000
params_1000 = {'learning_rate': 0.0538129866548988,
 'reg_lambda': 78.83024499918844,
 'reg_alpha': 1.1789981443997697,
 'subsample': 0.5850723307909039,
 'colsample_bytree': 0.24017852529032113,
 'max_depth': 5}

In [14]:
# n_estimators = 2000
params_2000 = {'learning_rate': 0.04353489504974672,
 'reg_lambda': 4.545379085259231e-05,
 'reg_alpha': 3.943406343157982e-06,
 'subsample': 0.7747461085845304,
 'colsample_bytree': 0.1223723907272958,
 'max_depth': 3}

In [15]:
# n_estimators = 5000
params_5000 = {'learning_rate': 0.023172693725536604,
 'reg_lambda': 0.0010005788328675566,
 'reg_alpha': 3.871785864631445e-06,
 'subsample': 0.998601648186681,
 'colsample_bytree': 0.23009931707478923,
 'max_depth': 5}

In [16]:
test_data.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,target_enc_cat0,target_enc_cat1,target_enc_cat2,target_enc_cat3,target_enc_cat4,target_enc_cat5,target_enc_cat6,target_enc_cat7,target_enc_cat8,target_enc_cat9
0,B,B,B,C,B,B,A,E,E,I,...,8.199411,8.162038,8.186756,8.185477,8.189992,8.199043,8.190005,8.191722,8.145356,8.158563
1,A,B,A,C,B,C,A,E,C,H,...,8.186798,8.162038,8.192303,8.185477,8.189992,8.221418,8.190005,8.191722,8.248797,8.195199
2,B,A,A,A,B,B,A,E,D,K,...,8.199411,8.219706,8.192303,8.232115,8.189992,8.199043,8.190005,8.191722,8.205808,8.232337
3,B,B,A,C,B,D,A,E,A,N,...,8.199411,8.162038,8.192303,8.185477,8.189992,8.171351,8.190005,8.191722,8.134364,8.210000
4,B,B,A,C,B,C,A,E,C,F,...,8.199411,8.162038,8.192303,8.185477,8.189992,8.221418,8.190005,8.191722,8.248797,8.206115


In [17]:
type(test_data)

pandas.core.frame.DataFrame

In [18]:
final_predictions = []
scores = []

for fold in range(5):
    
    # making training data and validating data for each fold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    
    X_test = test_data.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[test_data.columns]
    X_valid = X_valid[test_data.columns]
    
    ## One Hot Encoding for categorical data
    OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
    
    # fitting and transforming the training and test data
    OH_encoder.fit(X_train[cat_cols])
    OH_X_train = OH_encoder.transform(X_train[cat_cols])
    OH_X_valid = OH_encoder.transform(X_valid[cat_cols])
    OH_X_test = OH_encoder.transform(X_test[cat_cols])
    
    # Naming the one hot encoded columns
    OH_X_train = pd.DataFrame(OH_X_train, columns=[f"ohe_{i}" for i in range(OH_X_train.shape[1])])
    OH_X_valid = pd.DataFrame(OH_X_valid, columns=[f"ohe_{i}" for i in range(OH_X_valid.shape[1])])
    OH_X_test = pd.DataFrame(OH_X_test, columns=[f"ohe_{i}" for i in range(OH_X_test.shape[1])])
     
    # Adding one hot encoded columns to main data (training, validating, test)
    X_train = pd.concat([X_train, OH_X_train], axis=1)
    X_valid = pd.concat([X_valid, OH_X_valid], axis=1)
    X_test = pd.concat([X_test, OH_X_test], axis=1)
    
    # Dropping the categorical columns, as their one hot encoded columns are added to main data
    X_train = X_train.drop(cat_cols, axis=1)
    X_valid = X_valid.drop(cat_cols, axis=1)
    X_test = X_test.drop(cat_cols, axis=1)
    
    # making the model
    model = XGBRegressor(
        n_estimators=2000,
        **params_2000,
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        random_state=50,
    )
    # fitting the data in the model
    model.fit(X_train, y_train, early_stopping_rounds=300,
              eval_set=[(X_valid, y_valid)], verbose=1000)
    
    # getting valid predictions and test predictions
    preds_valid = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    
    # rmse on valid predictions
    rmse = mean_squared_error(y_valid, preds_valid, squared=False)
    scores.append(rmse)
    
    print(f"Fold: {fold}, RMSE: {rmse}, \nPredictions on test data: {test_preds}")

[0]	validation_0-rmse:7.43966
[1000]	validation_0-rmse:0.72029
[1999]	validation_0-rmse:0.71735
Fold: 0, RMSE: 0.7173544472729189, 
Predictions on test data: [8.072433  8.332329  8.451173  ... 8.4778805 8.187541  8.010138 ]
[0]	validation_0-rmse:7.44294
[1000]	validation_0-rmse:0.72819
[1999]	validation_0-rmse:0.72495
Fold: 1, RMSE: 0.7249523369628741, 
Predictions on test data: [8.052662  8.352034  8.462922  ... 8.468063  8.124349  7.9782763]
[0]	validation_0-rmse:7.43899
[1000]	validation_0-rmse:0.72368
[1999]	validation_0-rmse:0.72070
Fold: 2, RMSE: 0.7207026138886449, 
Predictions on test data: [8.080405  8.382168  8.450365  ... 8.44541   8.084091  7.9776425]
[0]	validation_0-rmse:7.44545
[1000]	validation_0-rmse:0.72277
[1999]	validation_0-rmse:0.71995
Fold: 3, RMSE: 0.7199525335027647, 
Predictions on test data: [8.02317   8.364025  8.41886   ... 8.382373  8.093361  7.9953537]
[0]	validation_0-rmse:7.44524
[1000]	validation_0-rmse:0.71930
[1999]	validation_0-rmse:0.71787
Fold: 4,

## Final predictions (`final_preds`) based on best parameters

In [19]:
final_predictions

[array([8.072433 , 8.332329 , 8.451173 , ..., 8.4778805, 8.187541 ,
        8.010138 ], dtype=float32),
 array([8.052662 , 8.352034 , 8.462922 , ..., 8.468063 , 8.124349 ,
        7.9782763], dtype=float32),
 array([8.080405 , 8.382168 , 8.450365 , ..., 8.44541  , 8.084091 ,
        7.9776425], dtype=float32),
 array([8.02317  , 8.364025 , 8.41886  , ..., 8.382373 , 8.093361 ,
        7.9953537], dtype=float32),
 array([8.053768 , 8.355371 , 8.44058  , ..., 8.4383745, 8.169567 ,
        8.045488 ], dtype=float32)]

In [20]:
final_preds = np.mean(np.column_stack(final_predictions), axis=1)
final_preds

array([8.056488, 8.357185, 8.44478 , ..., 8.44242 , 8.131782, 8.001379],
      dtype=float32)

## Storing the Final predictions (`final_preds`) in `hyperparameter optuna.csv`

In [21]:
output = pd.DataFrame({"Id" : sample_submission.id,"target" : final_preds})
output.to_csv('submission.csv', index=False)